In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import OrderedDict

In [2]:
df_policy = pd.read_csv('./policy_claim/policy_0702.csv')
df_claim= pd.read_csv('./policy_claim/claim_0702.csv')
df_train= pd.read_csv('training-set.csv')
df_test = pd.read_csv('testing-set.csv')

In [3]:
#group data by Policy _Number
policy_grp = df_policy.groupby("Policy_Number")
print("最大保單數為 : %s"%(max(policy_grp.size())))

最大保單數為 : 18


In [4]:
df_policy.head()

,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,...,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,...,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
1,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,...,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,...,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
3,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,...,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,Fcd3d3578e9ae55aa6140da432f69db2bd0c50fad,bd864927fc796b75c917ade554c0657d22b860bd,,3828f4a4c40ac91647919306d0de0d11043d89fe,de36049ef6a91e6dd1a031ab5501beca28cc3610,cca82bf061cfd670f460f9e4a1b9ca853378382b,2006,1341,10,...,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,


### columns need to be splited into two part
    * reserve(ID....)
    * autoencoder
        * ---int
        * ---one hot

In [5]:
df_policy.columns

Index(['Policy_Number', 'Insured's_ID', 'Prior_Policy_Number', 'Cancellation',
       'Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2', 'Manafactured_Year_and_Month',
       'Engine_Displacement_(Cubic_Centimeter)', 'Imported_or_Domestic_Car',
       'Coding_of_Vehicle_Branding_&_Type', 'qpt', 'fpt',
       'Main_Insurance_Coverage_Group', 'Insurance_Coverage',
       'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3',
       'Coverage_Deductible_if_applied', 'Premium',
       'Replacement_cost_of_insured_vehicle', 'Distribution_Channel',
       'Multiple_Products_with_TmNewa_(Yes_or_No?)', 'lia_class', 'plia_acc',
       'pdmg_acc', 'fassured', 'ibirth', 'fsex', 'fmarriage', 'aassured_zip',
       'iply_area', 'dbirth', 'fequipment1', 'fequipment2', 'fequipment3',
       'fequipment4', 'fequipment5', 'fequipment6', 'fequipment9',
       'nequipment9'],
      dtype='object')

In [6]:
# three type of idx
org_idx=df_policy.columns
autoencoder_idx=['Insurance_Coverage','Main_Insurance_Coverage_Group','fpt','Coverage_Deductible_if_applied','qpt',
              'Insured_Amount1','Insured_Amount2','Insured_Amount3','Premium']
one_hot_idx=['Insurance_Coverage','Main_Insurance_Coverage_Group','fpt']

In [7]:
# for test
df_filter= df_policy[df_policy['Policy_Number']=='55789b8f86893761c9aa9e7bf17938e737decc68'][rule_out_idx]
df_filter

NameError: name 'rule_out_idx' is not defined

In [ ]:
def one_hot_encode(df,idx):
        return pd.get_dummies(df[idx])

df_1=one_hot_encode(df_policy,'Insurance_Coverage')
df_2=one_hot_encode(df_policy,'Main_Insurance_Coverage_Group')
df_3=one_hot_encode(df_policy,'fpt')
final=pd.concat([df_policy,df_1,df_2,df_3],axis=1)

In [ ]:
final.head()

In [ ]:
need_encode=pd.concat([df_policy[['Policy_Number','Insured_Amount1','Insured_Amount2','Insured_Amount3','Premium','qpt','Coverage_Deductible_if_applied']],df_1,df_2,df_3],axis=1) 

In [ ]:
a=ord_dict['1a1f5f582bf1ff23467352e8f2fd31acefddc654']
df = pd.DataFrame(a)
df

In [ ]:
# Import PCA
from sklearn.decomposition import PCA
# 值采用前6000个样本来加速计算
X= train[:6000].values
del train
# 标准化数据
X_std = StandardScaler().fit_transform(X)

# Call the PCA method with 5 components. 
pca = PCA(n_components=5)
pca.fit(X_std)
X_5d = pca.transform(X_std)

# "Target"也要取前6000个
Target = target[:6000]

In [ ]:
policy_grp_2 = need_encode.groupby("Policy_Number")

In [ ]:
# zero padding
insured_dict={}
np_array= np.zeros(shape=[351273,18, 70])
max_insurance=18
uni_policy_number=list(set(df_policy['Policy_Number']))
count=0
for element in uni_policy_number:
    np_ary=policy_grp_2.get_group(element)[['Insured_Amount1', 'Insured_Amount2',
       'Insured_Amount3', 'Premium', 'qpt', 'Coverage_Deductible_if_applied',
       '00I', '01A', '01J', '02K', '03L', '04M', '05E', '05N', '06F', '07P',
       '08H', '09@', '09I', '10A', '12L', '14E', '14N', '15F', '15O', '16G',
       '16P', '18@', '18I', '20B', '20K', '25G', '26H', '27I', '29B', '29K',
       '32N', '33F', '33O', '34P', '35H', '36I', '37J', '40M', '41E', '41N',
       '42F', '45@', '46A', '47B', '51O', '55J', '56B', '56K', '57C', '57L',
       '65K', '66C', '66L', '67D', '68E', '68N', '70G', '70P', '71H', '72@',
       '竊盜', '車損', '車責', 'P']].values
    padding_len=max_insurance-np_ary.shape[0]
    np_ary_pad=np.pad(np_ary,((0,padding_len),(0,0)),'constant')
    insured_dict.update({element:np_ary_pad}) 
    #np_array=np.concatenate(np_array,np_ary_pad)
    np_array[count]+=np_ary_pad
    count+=1
    
ord_dict=OrderedDict(insured_dict)

In [ ]:
test_dict=list(ord_dict.items())[0:100] 

In [ ]:
test_dict[0]

In [ ]:
ord_dict_2=OrderedDict(test_dict)

In [ ]:
len(ord_dict_2)

In [390]:
save_as_pickled_object(ord_dict_2,'test_100.pickle')

In [384]:
ord_dict_2['fcf514daed0379d434c42983a295997e2dadfde0'].shape

(18, 70)

In [375]:
import sys
def save_as_pickled_object(obj, filepath):
    """
    This is a defensive way to write pickle.write, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj)
    n_bytes = sys.getsizeof(bytes_out)
    with open(filepath, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])


def try_to_load_as_pickled_object_or_None(filepath):
    """
    This is a defensive way to write pickle.load, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    try:
        input_size = os.path.getsize(filepath)
        bytes_in = bytearray(0)
        with open(filepath, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        obj = pickle.loads(bytes_in)
    except:
        return None
    return obj

In [376]:
save_as_pickled_object(ord_dict,'new_one.pickle')

In [372]:
with open('new_one.pickle', 'wb') as handle:
    pickle.dump(ord_dict, handle)
with open('new_one.pickle', 'rb') as handle:
    b = pickle.load(handle)

OSError: [Errno 22] Invalid argument

# To be continued...

In [ ]:
#df_train needs replace 'Insured_Amount1','Insured_Amount2','Insured_Amount3','Premium'  with autoencoder vector

In [69]:
df_filter3= df_claim[df_claim['Policy_Number']=='91dc13e4e24dc9e0e0940c63924109e4ff66e18a']
df_filter3

,Claim_Number,Nature_of_the_claim,Policy_Number,Driver's_Gender,Driver's_Relationship_with_Insured,DOB_of_Driver,Marital_Status_of_Driver,Accident_Date,Cause_of_Loss,Paid_Loss_Amount,paid_Expenses_Amount,Salvage_or_Subrogation?,Coverage,Vehicle_identifier,At_Fault?,"Claim_Status_(close,_open,_reopen_etc)",Deductible,Accident_area,number_of_claimants,Accident_Time
0,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,56705,0,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,3000,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
1,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,30000,700,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
2,954ff50c11e2c6f6bf91514fdcca98578128da21,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/01,17ba0791499db908433b80f37c5fbc89b870084b,1100,0,0,16G,86ada5177903be297607713f3e0128e7b3f8a839,100.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,08:00
253,67f7824613ccfdd6a6387f2c20297ed258749285,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/02,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,1000,0,00I,86ada5177903be297607713f3e0128e7b3f8a839,0.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,18:00
254,67f7824613ccfdd6a6387f2c20297ed258749285,1,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,1,3,06/1969,1,2015/02,bd307a3ec329e10a2cff8fb87480823da114f8f4,0,0,0,16P,86ada5177903be297607713f3e0128e7b3f8a839,0.0,1,0,356a192b7913b04c54574d18c28d46e6395428ab,1.981,18:00


In [68]:
df_filter2= df_train[df_train['Policy_Number']=='91dc13e4e24dc9e0e0940c63924109e4ff66e18a']
df_filter2

,Policy_Number,Next_Premium
2754,91dc13e4e24dc9e0e0940c63924109e4ff66e18a,16035


# new data

## start from here

In [297]:
import pandas as pd
import numpy as np
from collections import OrderedDict

In [298]:
df_policy = pd.read_csv('./policy_claim/policy_0702.csv')
df_claim= pd.read_csv('./policy_claim/claim_0702.csv')
df_train= pd.read_csv('training-set.csv')
df_test = pd.read_csv('testing-set.csv')
df_policy.head()
print(df_policy.shape)

(1747942, 41)


In [299]:
with open('Insured_Amount.pickle', 'rb') as handle:
    Insured_Amount = pickle.load(handle)
encode= np.load('encode.npy')

new_dict=[]
for key, value in Insured_Amount.items():
    new_dict.append(key)

encode_list=pd.DataFrame(encode,columns={'encode_01','encode_02','encode_03','encode_04','encode_05',
                                                                'encode_06','encode_07','encode_08','encode_09','encode_10'
                                        })
encode_list['Policy_Number']=new_dict
encode_list.head()

,encode_08,encode_03,encode_04,encode_07,encode_02,encode_06,encode_01,encode_05,encode_10,encode_09,Policy_Number
0,-18.095303,-34.823547,18.113970,-52.279613,22.280916,-50.745190,34.512726,13.356975,-57.793297,76.332184,fcf514daed0379d434c42983a295997e2dadfde0
1,-19.602621,-30.546492,4.128561,-40.883175,28.856615,-25.465271,6.348990,2.609125,-37.106327,71.575920,9a5b50a078e790ea8f5dc2e51e8bb31bb66c4106
2,11.803707,-51.822613,75.169418,-22.973011,45.030689,-106.213196,2.502214,61.991299,-94.301346,89.298485,2b6319c5f4a4f837d828b0c04c2b6af06df0be54
3,-20.038214,-68.283485,40.596539,-67.876091,51.417145,-87.781364,27.679592,22.826279,-79.087570,85.122833,57be8289c0f85e21e7897f0a3039013d8280ebc2
4,-44.110348,-21.763868,39.202858,-32.576340,34.940273,-26.782759,17.450106,33.273911,-44.317822,53.683498,cf5290877433ff1bed07d585d5e31797cea843c8


In [300]:
df_train.shape[0]+df_test.shape[0]

351273

In [302]:
# 'Insurance_Coverage','Main_Insurance_Coverage_Group', 'fpt' need to be rule out
columns_idx=df_policy.columns
rule_out_idx=['Insurance_Coverage','Coverage_Deductible_if_applied','Main_Insurance_Coverage_Group','fpt','qpt',
              'Insured_Amount1','Insured_Amount2','Insured_Amount3','Premium']
encode_columns_idx=[item for item in columns_idx if item not in rule_out_idx]

In [304]:
df_policy_with_encode=df_policy[encode_columns_idx]
print(df_policy.shape)
a=df_policy_with_encode.drop_duplicates(keep='first')
a.shape

(1747942, 41)


In [313]:
a.head()

,Policy_Number,Insured's_ID,Prior_Policy_Number,Cancellation,Vehicle_identifier,Vehicle_Make_and_Model1,Vehicle_Make_and_Model2,Manafactured_Year_and_Month,Engine_Displacement_(Cubic_Centimeter),Imported_or_Domestic_Car,...,iply_area,dbirth,fequipment1,fequipment2,fequipment3,fequipment4,fequipment5,fequipment6,fequipment9,nequipment9
0,79110176bf64b5094c19aad785aeac56e36cb609,Ae03a1ba75867f060504f8ee798d06275a5e5bb9c,e1379d9c360fe4a20027b8acc161ec93f2727e66,,a05c4dbcfc92d99e715441b9cd3fcab5ce117be5,7d1fedf43d62c640985e40c5628fea6d902c142b,566bb5a9ea0fa963636632a0d1754ea06da11c13,2010,1342,10,...,c1dfd96eea8cc2b62785275bca38ac261256e278,08/1984,0,0,0,0,0,0,0,
2,61a449f6180b23379a584af904d7412205d8ff8c,J7697af29d66cb8a160fb68b1412ef06716ca4a57,a2f57eef440f29783da32b4e3708e0ea5b84b9c4,,6d30feebf8bf0e46af73cd2a1d0649d4f0f83f8a,9e1816a8c2a63b7f96a82ce6487b2d43222750db,de17c3df362a74aef9d15b1c95ff5cf7ff94f628,2006,1794,10,...,fa35e192121eabf3dabf9f5ea6abdbcbc107ac3b,10/1975,0,0,0,0,0,0,0,
4,1a1f5f582bf1ff23467352e8f2fd31acefddc654,Fcd3d3578e9ae55aa6140da432f69db2bd0c50fad,bd864927fc796b75c917ade554c0657d22b860bd,,3828f4a4c40ac91647919306d0de0d11043d89fe,de36049ef6a91e6dd1a031ab5501beca28cc3610,cca82bf061cfd670f460f9e4a1b9ca853378382b,2006,1341,10,...,c1dfd96eea8cc2b62785275bca38ac261256e278,12/1989,0,0,0,0,0,0,0,
7,6b420a887376f4dce53e22e600f2ca56aadec0ee,P1cf232e92478ef7fd1aab3be9f94b3bfe812e422,872830c25ddf5ac8a050f25812bd7a14aa1eb9e5,,4450ffd7879ec507d2400f346466a9fbbb54d12a,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,eeb2ced6b829daf594d1245fc322ef3cd9bb69dc,2002,2995,40,...,356a192b7913b04c54574d18c28d46e6395428ab,01/1963,0,0,0,0,0,0,0,
8,15a5e6b7c385d23c01bf4eaa49f310d638d7d3d9,27a85951576aa3af5b9ed77f0015a2af9853488fe,9429254b6bc54ec448c6f7d664b00d04749034d6,,8a57375aeaee76be15a11544079e9b6c768a1d1a,7f913cfdeb239c0caed5fb6e4c8a6199ecb339fc,a2237ec5b04868622e576ed20bd7357a452a93d2,2001,2164,24,...,356a192b7913b04c54574d18c28d46e6395428ab,NaN,0,0,0,0,0,0,0,


In [285]:
if a.loc[0].any()==a.loc[1].any():
    print(True)

True


In [286]:
for idx in a.columns:
    print(a.loc[0:1][idx])

0    79110176bf64b5094c19aad785aeac56e36cb609
1    79110176bf64b5094c19aad785aeac56e36cb609
Name: Policy_Number, dtype: object
0    1551
1    1163
Name: Premium, dtype: int64
0    45.4
1    45.4
Name: Replacement_cost_of_insured_vehicle, dtype: float64
0    5037717f120cf25bbeea6a42e62676bc5c911b12
1    5037717f120cf25bbeea6a42e62676bc5c911b12
Name: Distribution_Channel, dtype: object
0    0.0
1    0.0
Name: Multiple_Products_with_TmNewa_(Yes_or_No?), dtype: float64
0    3
1    3
Name: lia_class, dtype: int64
0   -0.1
1   -0.1
Name: plia_acc, dtype: float64
0    0.0
1    0.0
Name: pdmg_acc, dtype: float64
0    1
1    1
Name: fassured, dtype: int64
0    08/1984
1    08/1984
Name: ibirth, dtype: object
0    2
1    2
Name: fsex, dtype: object
0    1
1    1
Name: fmarriage, dtype: object
0    9a70776c743352cfcf688e52512673332e5e4007
1    9a70776c743352cfcf688e52512673332e5e4007
Name: aassured_zip, dtype: object
0    c1dfd96eea8cc2b62785275bca38ac261256e278
1    c1dfd96eea8cc2b62785275bca38a

In [85]:
test.head()

,0,1,2,3,4,5,6,7,8,9
0,-18.095303,-34.823547,18.113970,-52.279613,22.280916,-50.745190,34.512726,13.356975,-57.793297,76.332184
1,-19.602621,-30.546492,4.128561,-40.883175,28.856615,-25.465271,6.348990,2.609125,-37.106327,71.575920
2,11.803707,-51.822613,75.169418,-22.973011,45.030689,-106.213196,2.502214,61.991299,-94.301346,89.298485
3,-20.038214,-68.283485,40.596539,-67.876091,51.417145,-87.781364,27.679592,22.826279,-79.087570,85.122833
4,-44.110348,-21.763868,39.202858,-32.576340,34.940273,-26.782759,17.450106,33.273911,-44.317822,53.683498
